#  AzureDay - Part-1: COVID-19 Data Analysis using .Net DataFrame API

### Dataset

- [2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE - Daily reports](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_daily_reports).

### Introduction 

[**DataFrame**](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/): DataFrame is a new type introduced in .Net. It is similar to DataFrame in Python which is used to manipulate data in notebooks. It's a collection of columns containing data similar to a table and very helpful in analyzing tabular data. It works flawlessly without creating types/classes mapped to columns in a table which we used to do with [ML.Net](https://dotnet.microsoft.com/apps/machinelearning-ai/ml-dotnet). It has support for GroupBy, Sort, Filter which makes analysis very handy. It's a in-memory representation of structured data.

In this tutorial we'll cover below features
- Load a CSV
- Metada
    - Description
    - Info
- Display records
    - Head
    - Sample
    - 
- Filtering
- Grouping
- Aggregate

For overview, please refer below links
- [An Introduction to DataFrame](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/)
- [Exploring the C# Dataframe API](https://www.youtube.com/watch?v=FI3VxXClJ7Y)



### Summary

Below is the summary of steps we'll be performing

1. Define application level items
    - Nuget packages
    - Namespaces
    - Constants
     
2. Utility Functions
    - Formatters    

3. Load Dataset
    - Download Dataset from [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data)
    - Load dataset in DataFrame
    
4. Analyse Data
    - Date Range
    - Display Datset - display(dataframe)
    - Display Top 5 Rows - dataframe.Head(5)
    - Display Random 6 Rows - dataframe.Sample(6)    
    - Display Dataset Statistics - dataframe.Description()
    - Display Dataset type information - dataframe.Info()

5. Data Cleaning
    - Remove Invalid cases

6. Data Visualization
    - Global
        - Confirmed Vs Deaths Vs Recovered
        - Top 5 Countries with Confirmed cases
        - Top 5 Countries with Death cases
        - Top 5 Countries with Recovered cases
    - Italy
        - Confirmed Vs Deaths Vs Recovered
        
**Note** : Graphs/Plots may not render in GitHub due to secutiry reasons, however if you run this notebook locally/binder they will render.

### 1. Define Application wide Items

#### Nuget Packages


In [1]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.Data.Analysis"

// Install XPlot package
#r "nuget:XPlot.Plotly"
    
// CSV Helper Package for reading CSV
#r "nuget:CsvHelper"

Installing package Microsoft.ML...................done!

Successfully added reference to package Microsoft.ML, version 1.5.1

Installing package Microsoft.Data.Analysis......done!

Successfully added reference to package Microsoft.Data.Analysis, version 0.4.0

Installing package XPlot.Plotly......done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

Installing package CsvHelper......done!

Successfully added reference to package CsvHelper, version 15.0.5

#### Namespaces

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.IO;
using System.Net.Http;
using System.Globalization;
using CsvHelper;
using CsvHelper.Configuration;
using XPlot.Plotly;

#### Constants

In [11]:
// Column Names
const string FIPS = "FIPS";
const string ADMIN = "Admin2";
const string STATE = "Province_State";
const string COUNTRY = "Country_Region";
const string LAST_UPDATE = "Last_Update";
const string LATITUDE = "Lat";
const string LONGITUDE = "Long_";
const string CONFIRMED = "Confirmed";
const string DEATHS = "Deaths";
const string RECOVERED = "Recovered";
const string ACTIVE = "Active";
const string COMBINED_KEY = "Combined_Key";

// File
const string DATASET_FILE = "07-13-2020";
const string FILE_EXTENSION = ".csv";
const string NEW_FILE_SUFFIX = "_new";
const char SEPARATOR = ',';
const char SEPARATOR_REPLACEMENT = '_';
const string DATASET_GITHUB_DIRECTORY = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/";

// DataFrame/Table
const int TOP_COUNT = 5;
const int DEFAULT_ROW_COUNT = 10;
const string VALUES = "Values";
const string ITALY = "Italy";

### 2. Utility Functions

#### Formatters

By default the output of DataFrame is not proper and in order to display it as a table, we need to have a custom formatter implemented as shown in next cell. 

In [12]:
// Formats the table

Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = DEFAULT_ROW_COUNT;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }

    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));

    writer.Write(t);
}, "text/html");

#### Copy dataset csv and replace Separator in cells

In [13]:
// Replace a characeter in a cell of csv with a defined separator
private void CreateCsvAndReplaceSeparatorInCells(string inputFile, string outputFile, char separator, char separatorReplacement)
{
    var culture = CultureInfo.InvariantCulture;
    using var reader = new StreamReader(inputFile);
    using var csvIn = new CsvReader(reader, new CsvConfiguration(culture));
    using var recordsIn = new CsvDataReader(csvIn);
    using var writer = new StreamWriter(outputFile);
    using var outCsv = new CsvWriter(writer, culture);

    // Write Header
    csvIn.ReadHeader();
    var headers = csvIn.Context.HeaderRecord;
    foreach (var header in headers)
    {
        outCsv.WriteField(header.Replace(separator, separatorReplacement));
    }
    outCsv.NextRecord();

    // Write rows
    while (recordsIn.Read())
    {
        var columns = recordsIn.FieldCount;
        for (var index = 0; index < columns; index++)
        {
            var cellValue = recordsIn.GetString(index);
            outCsv.WriteField(cellValue.Replace(separator, separatorReplacement));
        }
        outCsv.NextRecord();
    }
}

### 3. Load Dataset

#### Download Dataset from [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data)

We'll be using COVID-19 dataset from [Johns Hopkins CSSE](https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data). The **csse_covid_19_data directory** has .csv file for each day and we'll be performing analysis on latest file present. Latest file present at the time of last modification of this notebook was **06-10-2020.csv**. If you wish to use a different file, update **DATASET_FILE** constant in Constants cell above.

We'll download file to current directory.

In [15]:
// Download csv from github
var originalFileName = $"{DATASET_FILE}{FILE_EXTENSION}";
if (!File.Exists(originalFileName))
{
    var remoteFilePath = $"{DATASET_GITHUB_DIRECTORY}/{originalFileName}";
    display(remoteFilePath);
    var contents = new HttpClient()
        .GetStringAsync(remoteFilePath).Result;
        
    File.WriteAllText(originalFileName, contents);
}

#### Load dataset in DataFrame

**Issue**: We can load csv using LoadCsv(..) method of DataFrame. However, there is an [issue](https://github.com/dotnet/corefxlab/issues/2787) of not allowing quotes and separator(comma in this case) in a cell value. 
The dataset, we are using has both of them and LoadCsv fails for it. 
As a workaround, we'll use CSVHelper to read the csv file and replace command separator with underscore, save the file and use it to load in DataFrame LoadCsv(..) method.

<img src=".\assets\invalid-characters.png" alt="Invalid Character" style="zoom: 80%;" />

In [16]:
// Load and create a copy of dataset file
var newFileName = $"{DATASET_FILE}{NEW_FILE_SUFFIX}{FILE_EXTENSION}";
display(newFileName);
CreateCsvAndReplaceSeparatorInCells(originalFileName, newFileName, SEPARATOR, SEPARATOR_REPLACEMENT);

07-13-2020_new.csv

In [17]:
var covid19Dataframe = DataFrame.LoadCsv(newFileName);

### 4. Data Analysis

Data analysis is a critical activity in the field of Data science. It provides ways to uncover the hidden attributes of a dataset which can't be analyzed or predicted by simply looking at the data source. DataFrame makes the analysis simple by providing great API's such as GroupBy, Sort, Filter etc. Jupyter notebook is great tool for this kind of activity which maintains values of variables executed in a cell and providing it to other cells.

##### Finding the range in the records in Dataset

In DataFrame, Columns property allows access to values within a column by specifying column name. we'll use Last_Update column to get the date and sort it to get the start and end date

In [18]:
// Gets the data range

var dateRangeDataFrame = covid19Dataframe.Columns[LAST_UPDATE].ValueCounts();
var dataRange = dateRangeDataFrame.Columns[VALUES].Sort();
var lastElementIndex = dataRange.Length - 1;

var startDate = DateTime.Parse(dataRange[0].ToString()).ToShortDateString();
var endDate  = DateTime.Parse(dataRange[lastElementIndex].ToString()).ToShortDateString(); // Last Element

display(h4($"The data is between {startDate} and {endDate}"));

The data is between 7/6/2020 and 7/14/2020

##### Display 10 records

Here we have 12 columns which includes Country, State, Confirmed, Deaths, Recovered and Active cases

In [19]:
display(covid19Dataframe)

0,45001,Abbeville,South Carolina,US,2020-07-14 04:34:46,34.223335,-82.46171,157,1,0,156,Abbeville_ South Carolina_ US,640.1109,0.6369427
1,22001,Acadia,Louisiana,US,2020-07-14 04:34:46,30.295065,-92.4142,1383,45,0,1338,Acadia_ Louisiana_ US,2229.0273,3.253796
2,51001,Accomack,Virginia,US,2020-07-14 04:34:46,37.76707,-75.63235,1041,14,0,1027,Accomack_ Virginia_ US,3221.3145,1.3448607
3,16001,Ada,Idaho,US,2020-07-14 04:34:46,43.452656,-116.241554,4448,25,0,4423,Ada_ Idaho_ US,923.613,0.56205034
4,19001,Adair,Iowa,US,2020-07-14 04:34:46,41.330757,-94.47106,17,0,0,17,Adair_ Iowa_ US,237.69576,0
5,21001,Adair,Kentucky,US,2020-07-14 04:34:46,37.1046,-85.281296,142,19,0,123,Adair_ Kentucky_ US,739.5063,13.380281
6,29001,Adair,Missouri,US,2020-07-14 04:34:46,40.190586,-92.600784,100,0,0,100,Adair_ Missouri_ US,394.58627,0
7,40001,Adair,Oklahoma,US,2020-07-14 04:34:46,35.88494,-94.65859,137,4,0,133,Adair_ Oklahoma_ US,617.28394,2.919708
8,8001,Adams,Colorado,US,2020-07-14 04:34:46,39.87432,-104.33626,4788,158,0,4630,Adams_ Colorado_ US,925.35864,3.2999165
9,16003,Adams,Idaho,US,2020-07-14 04:34:46,44.893337,-116.45452,12,0,0,12,Adams_ Idaho_ US,279.45972,0


##### Display Top 5 records

In [14]:
covid19Dataframe.Head(5)

0,45001,Abbeville,South Carolina,US,2020-06-11 03:33:41,34.223335,-82.46171,60,0,0,60,Abbeville_ South Carolina_ US,244.62837,0
1,22001,Acadia,Louisiana,US,2020-06-11 03:33:41,30.295065,-92.4142,520,29,0,491,Acadia_ Louisiana_ US,838.1014,5.576923
2,51001,Accomack,Virginia,US,2020-06-11 03:33:41,37.76707,-75.63235,984,13,0,971,Accomack_ Virginia_ US,3044.9314,1.3211383
3,16001,Ada,Idaho,US,2020-06-11 03:33:41,43.452656,-116.241554,854,22,0,832,Ada_ Idaho_ US,177.33037,2.5761125
4,19001,Adair,Iowa,US,2020-06-11 03:33:41,41.330757,-94.47106,10,0,0,10,Adair_ Iowa_ US,139.82103,0


##### Display Random 6 records

In [15]:
covid19Dataframe.Sample(6)

0,55071,Manitowoc,Wisconsin,US,2020-06-11 03:33:41,44.11947,-87.80929,40,1,0,39,Manitowoc_ Wisconsin_ US,50.645092,2.5
1,48313,Madison,Texas,US,2020-06-11 03:33:41,30.966948,-95.93004,17,0,0,17,Madison_ Texas_ US,119.01428,0
2,<null>,,Karnataka,India,2020-06-11 03:33:41,14.70518,76.166435,5921,66,2604,3251,Karnataka_ India,8.763713,1.1146766
3,12045,Gulf,Florida,US,2020-06-11 03:33:41,29.93543,-85.24271,8,0,0,8,Gulf_ Florida_ US,58.655327,0
4,48041,Brazos,Texas,US,2020-06-11 03:33:41,30.663645,-96.302055,665,24,0,641,Brazos_ Texas_ US,290.1257,3.6090226
5,18127,Porter,Indiana,US,2020-06-11 03:33:41,41.45987,-87.06849,576,33,0,543,Porter_ Indiana_ US,338.05,5.7291665


##### Display Dataset Statistics such as Total, Max, Min, Mean of items in a column

In [16]:
covid19Dataframe.Description()

0,Length (excluding null values),3039,3645,3645,3717,3717,3717,3717,3645,3666
1,Max,99999,71.7069,178.065,290143,41128,533504,249015,12380.361,100
2,Min,0,-52.368,-164.03539,0,0,0,-583819,0,0
3,Mean,25640.164,35.37938,-70.273254,1980.1558,111.97229,929.4611,925.1859,339.88214,3.4311118


##### Display Dataset type information for each column

In [17]:
covid19Dataframe.Info()

0,DataType,System.Single,System.String,System.String,System.String,System.String,System.Single,System.Single,System.Single,System.Single,System.Single,System.Single,System.String,System.Single,System.Single
1,Length (excluding null values),3039,3717,3717,3717,3717,3645,3645,3717,3717,3717,3717,3717,3645,3666


### 5. Data Cleaning

Data Cleaning is another important activity in which remove the irrelevant data present in our dataset. This irrelevant data can be due missing values, invalid values or an outlier. The columns with less significance is removed for better analysis and prediction of our data. In order to keep this notebook simple, we'll use one of the techniques to remove invalid data. In this we are going to remove invalid Active cases such as the ones having negative values. The other techniques we can apply on data  could be DropNull to remove rows with null values, FillNull to fill null values with other such as mean, average. We can transform DataFrame and remove some of the unnecessary columns.

#### Remove invalid Active cases

In [18]:
covid19Dataframe.Description()

0,Length (excluding null values),3039,3645,3645,3717,3717,3717,3717,3645,3666
1,Max,99999,71.7069,178.065,290143,41128,533504,249015,12380.361,100
2,Min,0,-52.368,-164.03539,0,0,0,-583819,0,0
3,Mean,25640.164,35.37938,-70.273254,1980.1558,111.97229,929.4611,925.1859,339.88214,3.4311118


From the above description table, we could see negative value for Active cases which seems to be incorrect as number of active cases is cases is calculated by the below formula

**Active = Confirmed - Deaths - Recovered**

In order to check for invalid active cases, we'll use DataFrame **Filter** to retrive active column values whose value is less than 0.0

In [19]:
// Filter : Gets active records with negative calues

PrimitiveDataFrameColumn<bool> invalidActiveFilter = covid19Dataframe.Columns[ACTIVE].ElementwiseLessThan(0.0);
var invalidActiveDataFrame = covid19Dataframe.Filter(invalidActiveFilter);
display(invalidActiveDataFrame)

0,90004,Unassigned,Arizona,US,2020-06-11 03:33:41,<null>,<null>,0,2,0,-2,Unassigned_ Arizona_ US,<null>,<null>
1,90018,Unassigned,Indiana,US,2020-06-11 03:33:41,<null>,<null>,0,182,0,-182,Unassigned_ Indiana_ US,<null>,<null>
2,90024,Unassigned,Maryland,US,2020-06-11 03:33:41,<null>,<null>,0,43,0,-43,Unassigned_ Maryland_ US,<null>,<null>
3,90056,Unassigned,Wyoming,US,2020-06-11 03:33:41,<null>,<null>,0,17,0,-17,Unassigned_ Wyoming_ US,<null>,<null>
4,<null>,,Cantabria,Spain,2020-06-11 03:33:41,43.1828,-3.9878,2324,202,2287,-165,Cantabria_ Spain,399.55917,8.691911
5,<null>,,Ceuta,Spain,2020-06-11 03:33:41,35.8894,-5.3213,163,4,163,-4,Ceuta_ Spain,192.15128,2.4539878
6,<null>,,Diamond Princess,Canada,2020-06-11 03:33:41,<null>,<null>,0,1,0,-1,Diamond Princess_ Canada,<null>,<null>
7,<null>,,Extremadura,Spain,2020-06-11 03:33:41,39.4937,-6.0679,2982,508,2652,-178,Extremadura_ Spain,279.88858,17.035547
8,<null>,,Galicia,Spain,2020-06-11 03:33:41,42.5751,-8.1339,9153,609,9204,-660,Galicia_ Spain,338.94464,6.6535563
9,<null>,,Hiroshima,Japan,2020-06-11 03:33:41,34.60531,132.78871,165,3,164,-2,Hiroshima_ Japan,5.8840795,1.8181819


If we take any record(index 5) and apply above formula to calculate 

**Active(-17) = Confirmed(0) - Deaths(17) - Recovered(0)**

We could see invalid active cases.

In order to remove it, we'll apply a **Filter** to DataFrame to get active values greater than or equal to 0.0. 

In [20]:
// Remove invalid active cases by applying filter

PrimitiveDataFrameColumn<bool> activeFilter = covid19Dataframe.Columns[ACTIVE].ElementwiseGreaterThanOrEqual(0.0);
covid19Dataframe = covid19Dataframe.Filter(activeFilter);
display(covid19Dataframe);

0,45001,Abbeville,South Carolina,US,2020-06-11 03:33:41,34.223335,-82.46171,60,0,0,60,Abbeville_ South Carolina_ US,244.62837,0
1,22001,Acadia,Louisiana,US,2020-06-11 03:33:41,30.295065,-92.4142,520,29,0,491,Acadia_ Louisiana_ US,838.1014,5.576923
2,51001,Accomack,Virginia,US,2020-06-11 03:33:41,37.76707,-75.63235,984,13,0,971,Accomack_ Virginia_ US,3044.9314,1.3211383
3,16001,Ada,Idaho,US,2020-06-11 03:33:41,43.452656,-116.241554,854,22,0,832,Ada_ Idaho_ US,177.33037,2.5761125
4,19001,Adair,Iowa,US,2020-06-11 03:33:41,41.330757,-94.47106,10,0,0,10,Adair_ Iowa_ US,139.82103,0
5,21001,Adair,Kentucky,US,2020-06-11 03:33:41,37.1046,-85.281296,101,19,0,82,Adair_ Kentucky_ US,525.9869,18.811882
6,29001,Adair,Missouri,US,2020-06-11 03:33:41,40.190586,-92.600784,74,0,0,74,Adair_ Missouri_ US,291.99384,0
7,40001,Adair,Oklahoma,US,2020-06-11 03:33:41,35.88494,-94.65859,95,4,0,91,Adair_ Oklahoma_ US,428.0436,4.2105265
8,8001,Adams,Colorado,US,2020-06-11 03:33:41,39.87432,-104.33626,3643,143,0,3500,Adams_ Colorado_ US,704.06885,3.9253364
9,16003,Adams,Idaho,US,2020-06-11 03:33:41,44.893337,-116.45452,3,0,0,3,Adams_ Idaho_ US,69.86493,0


**As seen above, negative active cases have been removed**

### 6. Visualization

Visualization of data helps business owners make better decisions. The DataFrame maintains data in a tabular format. In order to prepare data for different plots, I have used DataFrame features such as Sum, GroupBy, OrderBy, OrderByDescending etc. 

For visualization, I have used open source library called as [XPlot.Plotly](https://fslab.org/XPlot/plotly.html). Different plots have been used such as Bar, Pie and Line/Scatter Graph. 

#### Global

##### Collect Data

In [23]:
//  Gets the collection of confirmed, deaths and recovered

var confirmed = covid19Dataframe.Columns[CONFIRMED];
var deaths = covid19Dataframe.Columns[DEATHS];
var recovered = covid19Dataframe.Columns[RECOVERED];
var actives = covid19Dataframe.Columns[ACTIVE];

// Gets the sum of collection by using Sum method of DataFrame
var totalConfirmed = Convert.ToDouble(confirmed.Sum());
var totalDeaths = Convert.ToDouble(deaths.Sum());
var totalRecovered = Convert.ToDouble(recovered.Sum());
var totalActives = Convert.ToDouble(actives.Sum());

##### Confirmed Vs Deaths Vs Receovered cases

In [25]:
display(Chart.Plot(
    new Graph.Pie()
    {
        values = new double[]{totalConfirmed, totalDeaths, totalRecovered,totalActives},
        labels = new string[] {CONFIRMED, DEATHS, RECOVERED,ACTIVE}
    }
));

##### Top 5 Countries with Confirmed cases

In order to get top 5 countries data, I have used DataFrame's GroupBy, Sum, OrderByDescending methods

In [79]:
// The data for top 5 countries is not present in the csv file.
// In order to get that, first DataFrame's GROUPBY is used aginst the country.
// Then it was aggregated using SUM on Confirmed column.
// In the last, ORDERBYDESCENDING is used to get the top five countries.

var countryConfirmedGroup = covid19Dataframe.GroupBy(COUNTRY).Sum(CONFIRMED).OrderByDescending(CONFIRMED);
var topCountriesColumn = countryConfirmedGroup.Columns[COUNTRY];
var topConfirmedCasesByCountry = countryConfirmedGroup.Columns[CONFIRMED];

HashSet<string> countries = new HashSet<string>(TOP_COUNT);
HashSet<long> confirmedCases = new HashSet<long>(TOP_COUNT);
for(int index = 0; index < TOP_COUNT; index++)
{
    countries.Add(topCountriesColumn[index].ToString());
    confirmedCases.Add(Convert.ToInt64(topConfirmedCasesByCountry[index]));
}

In [80]:
var title = "Top 5 Countries : Confirmed";
var series1 = new Graph.Bar{
        x = countries.ToArray(),
        y = confirmedCases.ToArray()
    };

var chart = Chart.Plot(new []{series1});
chart.WithTitle(title);
display(chart);

##### Top 5 Countries with Deaths

In [81]:
// Get the data
var countryDeathsGroup = covid19Dataframe.GroupBy(COUNTRY).Sum(DEATHS).OrderByDescending(DEATHS);
var topCountriesColumn = countryDeathsGroup.Columns[COUNTRY];
var topDeathCasesByCountry = countryDeathsGroup.Columns[DEATHS];

HashSet<string> countries = new HashSet<string>(TOP_COUNT);
HashSet<long> deathCases = new HashSet<long>(TOP_COUNT);
for(int index = 0; index < TOP_COUNT; index++)
{
    countries.Add(topCountriesColumn[index].ToString());
    deathCases.Add(Convert.ToInt64(topDeathCasesByCountry[index]));
}

In [82]:
var title = "Top 5 Countries : Deaths";
var series1 = new Graph.Bar{
        x = countries.ToArray(),
        y = deathCases.ToArray()
    };

var chart = Chart.Plot(new []{series1});
chart.WithTitle(title);
display(chart);

##### Top 5 Countries with Recovered cases

In [83]:
// Get the data
var countryRecoveredGroup = covid19Dataframe.GroupBy(COUNTRY).Sum(RECOVERED).OrderByDescending(RECOVERED);
var topCountriesColumn = countryRecoveredGroup.Columns[COUNTRY];
var topRecoveredCasesByCountry = countryRecoveredGroup.Columns[RECOVERED];

HashSet<string> countries = new HashSet<string>(TOP_COUNT);
HashSet<long> recoveredCases = new HashSet<long>(TOP_COUNT);
for(int index = 0; index < TOP_COUNT; index++)
{
    countries.Add(topCountriesColumn[index].ToString());
    recoveredCases.Add(Convert.ToInt64(topRecoveredCasesByCountry[index]));
}

In [84]:
var title = "Top 5 Countries : Recovered";
var series1 = new Graph.Bar{
        x = countries.ToArray(),
        y = recoveredCases.ToArray()
    };

var chart = Chart.Plot(new []{series1});
chart.WithTitle(title);
display(chart);

#### Italy

##### Confirmed Vs Deaths Vs Receovered cases

Filering on Country column with ITALY as value

In [85]:
// Filering on Country column with ITALY as value

PrimitiveDataFrameColumn<bool> italyFilter = covid19Dataframe.Columns[COUNTRY].ElementwiseEquals(ITALY);
var italyDataFrame = covid19Dataframe.Filter(italyFilter);
            
var italyConfirmed = italyDataFrame.Columns[CONFIRMED];
var italyDeaths = italyDataFrame.Columns[DEATHS];
var italyRecovered = italyDataFrame.Columns[RECOVERED];
var italyActives = italyDataFrame.Columns[ACTIVE];

var italyTotalConfirmed = Convert.ToDouble(italyConfirmed.Sum());
var italyTotalDeaths = Convert.ToDouble(italyDeaths.Sum());
var italyTotalRecovered = Convert.ToDouble(italyRecovered.Sum());
double italyTotalActive = Convert.ToDouble(italyActives.Sum());

In [86]:
display(Chart.Plot(
    new Graph.Pie()
    {
        values = new double[]{italyTotalConfirmed, italyTotalDeaths, italyTotalRecovered,italyTotalActive},
        labels = new string[] {CONFIRMED, DEATHS, RECOVERED, ACTIVE}
    }
));

In [87]:
display("Total Actives : "+italyTotalActive)

Total Actives : 31710

## References
- [Using ML.NET in Jupyter notebooks](https://devblogs.microsoft.com/cesardelatorre/using-ml-net-in-jupyter-notebooks/)
- [An Introduction to DataFrame](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/)
- [DataFrame - Sample](https://github.com/dotnet/interactive/blob/master/NotebookExamples/csharp/Samples/HousingML.ipynb)
- [Getting started with ML.NET in Jupyter Notebooks](https://xamlbrewer.wordpress.com/2020/02/20/getting-started-with-ml-net-in-jupyter-notebooks/)
- [Tips and tricks for C# Jupyter notebook](https://ewinnington.github.io/posts/jupyter-tips-csharp)
- [Jupyter notebooks with C# and R running](https://github.com/ewinnington/noteb)
- [Data analysis using F# and Jupyter notebook — Samuele Resca](https://medium.com/@samueleresca/data-analysis-using-f-and-jupyter-notebook-samuele-resca-66a229e25306)
- [Exploring the C# Dataframe API](https://www.youtube.com/watch?v=FI3VxXClJ7Y) by Jon Wood
- [Coronavirus-COVID-19-Visualization-Prediction](https://www.kaggle.com/therealcyberlord/coronavirus-covid-19-visualization-prediction)
- [Data Modelling & Analysing Coronavirus (COVID19) Spread using Data Science & Data Analytics in Python Code](https://in.springboard.com/blog/data-modelling-covid/)